# Evidencia 2 Movilidad Urbana
## Parte 1: Modelado mediante sistemas multiagentes 

- Ruy Guzmán Camacho | A01639912
- Adrián Becerra Meza | A01639813
- Santiago González de Cosío Romero | A01640329

#### Librerías


In [187]:
#model design
import agentpy as ap

#visualizations
import matplotlib.pyplot as plt
import IPython
import random

### Definición de modelo

In [188]:
pos = [1, 2, 3, 4]
moves = [(0,-1), (1, 0)]


class StreetModel(ap.Model):

    def makeCircle(self,startX,startY,r):
        map = []
        Epsilon = 2.2
        for y in range(startX-r,startX+r+1):
            for x in range(startY-r,startY+r+1):
                # see if we're close to (x-a)**2 + (y-b)**2 == r**2
                if abs((x-startX)**2 + (y-startY)**2 - r**2) < Epsilon**2:
                    map.append((x,y))
        return map

    def resetNeighbors(self,carro):
        if len(self.carTable[carro.id]) != 0:
            self.carTable[carro.id][0].condition = 0
            self.carTable[carro.id].pop(0)

    def setup(self):

        self.carTable = {}

        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)
        self.carros = ap.AgentList(self, self.p['trafico'])

        

        
        # Create grid (street)
        self.street = ap.Grid(self, [self.p.size]*2, track_empty=True)
        #self.street.add_agents(trees, random=True, empty=True)
        self.street.add_agents(self.carros,[(2,21)]*self.p['trafico'])
        #self.street.add_agents(trees, [(0,21),(13,21)])
        range
        # Initiate a dynamic variable for all elements
        # Condition 0: trees, 1: car, 2: street 
        # position 0: top, 1: left
        # Draw trees

        #Initial position of number of cars available for traffic simulation
        pos_count = self.p.size -4
        self.carros.position = 0
        self.carros.inside = 0
        self.carros.id = 0
        self.isChoque = False
        
        id = 0

        
        for carro in self.carros:
            
            carro.id = id
            id += 1
            self.carTable[carro.id] = []
            carro.position = random.choice(pos)
            self.street.move_to(carro, (37, pos_count))
            pos_count -= 5
        
        self.agents.condition = 0
    
        # Draw Street
        routePoints = self.makeCircle(int(self.p.size/2),int(self.p.size/2), 7)
        self.road = routePoints
        self.roundabout = ap.AgentList(self,len(routePoints)*2)

        self.street.add_agents(self.roundabout,routePoints)
        
        self.roundabout.condition = 0
        self.roundabout.exit = -1
        
        #exit setup
        self.roundabout[1].exit = 0
        self.roundabout[21].exit = 3
        self.roundabout[18].exit = 1
        self.roundabout[38].exit = 2
        
        # draw cars
        self.carros.condition = 4
        self.carros.destination = -1

    def step(self):

        # Select cars
        carros = self.carros
        listcars = list(carros)
        road = self.road
        positions = self.street.positions
        
        startPoint = (37, self.p.size -3)

        # para que siga la ruta
        # move cars depending on position
        for carro in carros:
            
            #Remaining cars movement until StartPosition into simulation
            if (positions[carro]) != startPoint and carro.inside == 0:
                self.street.move_to(carro,(positions[carro][0], positions[carro][1] + 1))
                if (positions[carro]) == startPoint:
                    carro.inside = 1
                    #set car destination
                    carro.destination = random.choice(range(0,4))

                    #Car spawn in correct position
                    if carro.position == 1:
                        self.street.move_to(carro,(2,19))
                    elif carro.position == 2:
                        self.street.move_to(carro, (21,2))
                    elif carro.position == 3:
                        self.street.move_to(carro, (34, 21))
                    elif carro.position == 4:
                        self.street.move_to(carro, (19,34))        
            
            else:
                
                ActPos = positions[carro]


                neighbors = self.street.neighbors(carro)
                listNeighbor = list(neighbors)
                if carro.position == 1 or carro.position == 4: listNeighbor.reverse()


                if listNeighbor:
                    
                    for neighbor in listNeighbor:
                        if neighbor.id in self.carTable or neighbor.condition == 1:
                            self.isChoque = True 

                    
                    if self.isChoque == False or carro.inside == 2:
                        for neighbor in listNeighbor:
                            if neighbor.condition == 0:
                                if neighbor.exit == carro.destination:
                                    self.street.move_to(carro, (37, 20))
                                    carro.inside = 0
                                    self.resetNeighbors(carro)
                                    break
                                else:
                                    #print(neighbor)
                                    self.street.move_to(carro, positions[neighbor])
                                    neighbor.condition = 1
                                    self.carTable[carro.id].append(neighbor)     
                                    carro.inside = 2
                                    
                        
                        self.resetNeighbors(carro)
                        self.isChoque = False 
                            
                            

                    else:
                        print("Is For Me?")

                #bajara hasta que se tope con la glorieta                
                else:
                    if carro.position == 1:
                        self.street.move_by(carro, (1,0))
                    elif carro.position == 2:
                        self.street.move_by(carro, (0,1))
                    elif carro.position == 3:
                        self.street.move_by(carro, (-1,0))
                    elif carro.position == 4:
                        self.street.move_by(carro, (0,-1))
                
            burning_trees = self.agents.select(self.agents.condition == 0)
            if len(burning_trees) == 0:
                self.stop()


    def end(self):
        
        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))
                    

### Parametros

In [189]:
# Define parameters

parameters = {
    'Tree density': 1, # Percentage of grid covered by trees
    'size': 40,
    'steps': 100,
    'trafico': 5
    }


### Renderizado

In [190]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#808080', None:'#d5e5d5', 4:'#800080'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a roundabout\n"
                 f"Time-step: {model.t}, Cars: "
                 f"{len(model.carros)}")
    

fig, ax = plt.subplots()

circle1 = plt.Circle((20,20), 10, color='r')

ax.add_patch(circle1)
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=20))

Is For Me?
Is For Me?
Is For Me?
Is For Me?
